In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
6,First Responders in Mass. to Receive COVID-19 ...,https://www.nbcboston.com/news/local/first-res...,"Sun, 10 Jan 2021 17:13:47 GMT",2021-01-10 17:13:47+00:00
2,"MDH: 2,165 new COVID-19 cases, 44 deaths Sunda...",https://kstp.com/coronavirus/mdh-2165-new-covi...,"Sun, 10 Jan 2021 16:31:09 GMT",2021-01-10 16:31:09+00:00
9,Post-COVID heart concerns in young athletes: W...,https://www.abc27.com/news/post-covid-heart-co...,"Sun, 10 Jan 2021 16:20:00 GMT",2021-01-10 16:20:00+00:00
1,"Jan. 10: Iowa reports 1,330 new COVID-19 cases...",https://ktiv.com/2021/01/10/jan-10-iowa-report...,"Sun, 10 Jan 2021 16:16:36 GMT",2021-01-10 16:16:36+00:00
3,"DHEC: 3,667 new COVID-19 cases, 48 additional ...",https://www.wistv.com/2021/01/10/dhec-new-covi...,"Sun, 10 Jan 2021 16:12:00 GMT",2021-01-10 16:12:00+00:00
4,"Virginia COVID-19 Jan. 10 update: Over 5,100 n...",https://www.wavy.com/news/health/coronavirus/v...,"Sun, 10 Jan 2021 15:44:00 GMT",2021-01-10 15:44:00+00:00
5,U.S. sets another record for daily Covid-19 ca...,https://www.nbcnews.com/news/us-news/u-s-sets-...,"Sun, 10 Jan 2021 05:24:00 GMT",2021-01-10 05:24:00+00:00
8,California reports record 695 COVID-19 deaths ...,https://ktla.com/news/local-news/california-re...,"Sat, 09 Jan 2021 20:52:00 GMT",2021-01-09 20:52:00+00:00
7,Louisiana Department of Health announces COVID...,https://ldh.la.gov/index.cfm/newsroom/detail/5980,"Sat, 09 Jan 2021 20:14:03 GMT",2021-01-09 20:14:03+00:00
0,"279 additional Maine COVID-19 cases, no additi...",https://www.newscentermaine.com/article/news/h...,"Sat, 09 Jan 2021 14:55:00 GMT",2021-01-09 14:55:00+00:00
